In [96]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
%matplotlib inline 
import os
import gc
import time 
from math import log
path = "/home/ec2-user/Sanjay/main/credit_risk/data/competitions/home-credit-default-risk/"
import lightgbm as lgbm
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [97]:
data = pd.read_csv(path + 'application_train.csv')
test = pd.read_csv(path + 'application_test.csv')
sample_submission = pd.read_csv(path + 'sample_submission.csv')
y = data['TARGET']
del data['TARGET']
print('Shapes : ', data.shape, test.shape, sample_submission.shape)

Shapes :  (307511, 121) (48744, 121) (48744, 2)


In [98]:
categorical_feats = [f for f in data.columns if data[f].dtype == 'object']
for f_ in categorical_feats:
    data[f_], indexer = pd.factorize(data[f_])
    test[f_] = indexer.get_indexer(test[f_])

In [99]:
avg_buro = pd.read_csv(path + 'features/' + 'avg_buro.csv')
avg_prev = pd.read_csv(path + 'features/' + 'avg_prev.csv')
avg_pos = pd.read_csv(path + 'features/' + 'avg_pos.csv')
avg_cc_bal = pd.read_csv(path + 'features/' + 'avg_cc_bal.csv')
avg_inst = pd.read_csv(path + 'features/' + 'avg_inst.csv')

In [100]:
data = data.merge(right=avg_buro, how='left', on='SK_ID_CURR')  
data = data.merge(right=avg_prev, how='left', on='SK_ID_CURR')
data = data.merge(right=avg_pos, how='left', on='SK_ID_CURR')
data = data.merge(right=avg_cc_bal, how='left', on='SK_ID_CURR')
data = data.merge(right=avg_inst, how='left', on='SK_ID_CURR')

test = test.merge(right=avg_buro, how='left', on='SK_ID_CURR') 
test = test.merge(right=avg_prev, how='left', on='SK_ID_CURR')
test = test.merge(right=avg_pos, how='left', on='SK_ID_CURR')
test = test.merge(right=avg_cc_bal, how='left', on='SK_ID_CURR')
test = test.merge(right=avg_inst, how='left', on='SK_ID_CURR')
gc.collect(), data.shape, test.shape

(22590, (307511, 380), (48744, 380))

In [101]:
data = data.fillna(0)
test = test.fillna(0)
for i in data.columns:
    if data[i].describe()[1] > 1000 and data[i].describe()[3]>=0 and test[i].describe()[3]>=0:
        data[i] = data[i].map(lambda x : log(1+x))
        test[i] = test[i].map(lambda x : log(1+x))

### 5 Fold CV

In [102]:
parameters = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 50
}
iteration = 6000
vrbse = 400

In [103]:
def lgbm_model(X_tr, y_tr, X_va, y_va, fold, test):
    tr_data = lgb.Dataset(X_tr, label=y_tr, categorical_feature=categorical_feats)
    va_data = lgb.Dataset(X_va, label=y_va, categorical_feature=categorical_feats, reference=tr_data)
    
    model = lgb.train(parameters,
                      tr_data,
                      valid_sets=va_data,
                      num_boost_round=iteration,
                      early_stopping_rounds=120,
                      verbose_eval=vrbse)
    
    test_pred = model.predict(test)
    test_final = pd.DataFrame(test_pred, columns=['prob'+str(fold)])
    
    vald_pred = model.predict(X_va)
    y_va = y_va.reset_index(drop=True)
    valid = pd.DataFrame(vald_pred, columns=['prob'])
    valid_final = valid.join(y_va)
    
    return test_final, valid_final


In [104]:
start_time = time.time()
from sklearn.model_selection import KFold
data1 = data.drop(['SK_ID_CURR'], axis=1)
test1 = test.drop(['SK_ID_CURR'], axis=1)
X = data1
kf = KFold(n_splits=5, shuffle=True, random_state=2017)
kf.get_n_splits(X), kf.split(X)

test_final = sample_submission[['SK_ID_CURR']]
oof_data = pd.DataFrame(columns=['SK_ID_CURR', 'TARGET', 'prob'])

fold=1
for train_idx, test_idx in kf.split(X):
    print("FOLD ", fold, " -----------------------------")
    X_tr = X[X.index.isin(train_idx)]
    y_tr = y[y.index.isin(train_idx)]
    X_va = X[X.index.isin(test_idx)]
    y_va = y[y.index.isin(test_idx)]
    valid_final = data[['SK_ID_CURR']][data.index.isin(test_idx)].reset_index(drop=True)
    print(X_tr.shape, X_va.shape, y_tr.shape, y_va.shape, oof_data.shape)
    
    test, valid = lgbm_model(X_tr, y_tr, X_va, y_va, fold, test1)
    
    test_final = test_final.join(test)
    oof_data = oof_data.append(valid_final.join(valid))
    
    print(test_final.shape, test.shape, oof_data.shape, valid.shape)
    fold = fold+1
    
print(time.time() - start_time)

FOLD  1  -----------------------------
(246008, 379) (61503, 379) (246008,) (61503,) (0, 3)


/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 120 rounds.
[400]	valid_0's binary_logloss: 0.322378
[800]	valid_0's binary_logloss: 0.272602
[1200]	valid_0's binary_logloss: 0.25812
[1600]	valid_0's binary_logloss: 0.251498
[2000]	valid_0's binary_logloss: 0.247901
[2400]	valid_0's binary_logloss: 0.245214
[2800]	valid_0's binary_logloss: 0.24442
Early stopping, best iteration is:
[2708]	valid_0's binary_logloss: 0.244342
(48744, 2) (48744, 1) (61503, 3) (61503, 2)
FOLD  2  -----------------------------
(246009, 379) (61502, 379) (246009,) (61502,) (61503, 3)
Training until validation scores don't improve for 120 rounds.
[400]	valid_0's binary_logloss: 0.320753
[800]	valid_0's binary_logloss: 0.270367
[1200]	valid_0's binary_logloss: 0.255679
[1600]	valid_0's binary_logloss: 0.248735
[2000]	valid_0's binary_logloss: 0.245059
[2400]	valid_0's binary_logloss: 0.242168


KeyboardInterrupt: 

In [ ]:
test_final['TARGET'] = test_final.iloc[:,1:].apply(lambda x : x.mean(), axis=1)

In [ ]:
test_final[['SK_ID_CURR', 'TARGET']].to_csv(path + 'submit/' + 'lgbm4_dart.csv', index=False)
test_final.to_csv(path + 'submit/' + 'lgbm4_all5_dart.csv', index=False)
oof_data.to_csv(path + 'submit/' + 'oof4_dart.csv', index=False)

### Ensemble

In [106]:
import pandas as pd
data = pd.read_csv(path + 'submit/' + 'lgbm4_all5.csv')

In [107]:
for i in range(1,6):
    data['rank'+str(i)] = data['prob'+str(i)].rank(ascending=False)

In [109]:
data['Average'] = data.iloc[:,7:].apply(lambda x: x.mean(), axis=1)

In [111]:
data['Scaled_Rank'] = (data['Average'] - data['Average'].min()) / (data['Average'].max() - data['Average'].min())

In [112]:
data.head()

,SK_ID_CURR,prob1,prob2,prob3,prob4,prob5,TARGET,rank1,rank2,rank3,rank4,rank5,Average,Scaled_Rank
0,100001,0.028665,0.035021,0.037873,0.034678,0.026641,0.032576,33160.0,29202.0,28935.0,28875.0,34709.0,30976.2,0.635546
1,100005,0.147218,0.128987,0.145841,0.186561,0.145623,0.150846,5845.0,7324.0,6113.0,3822.0,5958.0,5812.4,0.119221
2,100013,0.025891,0.032717,0.030674,0.026015,0.037538,0.030567,35102.0,30571.0,33356.0,34587.0,27817.0,32286.6,0.662433
3,100028,0.048081,0.049728,0.034278,0.033290,0.034224,0.039920,22766.0,22288.0,31023.0,29716.0,29757.0,27110.0,0.556217
4,100038,0.154589,0.172926,0.182148,0.157157,0.135714,0.160507,5387.0,4516.0,4134.0,5195.0,6695.0,5185.4,0.106356


In [93]:
data['TARGET'] = data.iloc[:,7:].apply(lambda x : x.mean(), axis=1)

In [95]:
data[['SK_ID_CURR', 'TARGET']].to_csv(path + 'submit/' + 'lgbm4_rank_normalized.csv', index=False)